In [200]:
import pandas as pd
import ast
import re
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TextClassificationPipeline, pipeline
import transformers
from tqdm import tqdm
# Load the data
df  = pd.read_csv('./data/processed/cleaned_jobs.csv', sep=';')
print(len(df))
cluster = pd.read_csv('./csv_files/similarity.csv', sep=',')
#pipe = pipeline("token-classification", model="jobbert_skill_extraction")

token_skill_classifier = pipeline(model="jjzha/jobbert_skill_extraction", aggregation_strategy="first")
token_knowledge_classifier = pipeline(model="jjzha/jobbert_knowledge_extraction", aggregation_strategy="first")


1934


In [203]:
#for id in cluster.loc[cluster['cluster_kmean'] == 3]['id']:
    #print(id)
    
    #print(df.loc[df['id'] == id]['description'])
df.iloc[15]['description']

'the job board member technology company area ai role des  strong  cription this parttime onsite role based gilleleje president alfagate the president responsible overseeing daytoday operation company including developing implementing business strategy managing staff ensuring company meet financial goal the president also responsible representing company external stakeholder including client investor regulatory agency qualifications proven experience president similar executive leadership role strong strategic planning business development skill ability manage motivate team excellent communication interpersonal skill ability identify mitigate risk strong financial acumen experience food beverage industry plus bachelor s master s degree business administration related field'

In [190]:
cluster.loc[cluster['cluster_graph'] == 1]['id']

9       3737301784
51      3736863213
70      3728887478
91      3670459911
162     3727086963
163     3733213888
188     3606883718
203     3734719627
291     3728284107
409     3728426360
411     3706995810
448     3712727254
527     3691765287
575     3742061238
667     3707080426
1021    3742816391
1023    3741336412
1039    3744880691
1052    3744807778
1060    3744882176
1078    3737862110
1101    3714331463
1125    3744813137
Name: id, dtype: int64

In [212]:
for id in cluster.loc[cluster['cluster_graph'] == 10]['id']:
    description = len(df.loc[df['id'] == id]['description'].values[0])
    print(id)
    print(df.loc[df['id'] == id]['description'].values[0])
    print(description)
    if description < 3500:
        print(token_skill_classifier(df.loc[df['id'] == id]['description'].values[0]))

3661674712
signaloid provides computing platform quantify transmit track data uncertainty dynamically throughout computation unmodified computing workload our technology used application including material modeling autonomous system computational finance machine learning noisy intermediatescale quantum computing our team consists contrarian engineer combined research engineering leadership experience apple arc arm bell labs cmu university cambridge epfl ibm research max planck mit nec labs find http signaloidcom try signaloid uncertaintytracking computing platform signing free http getsignaloidio a driven respectful environment our motto we re going die  we never speak ill others even differ viewpoint show every day sense urgency treat respect though day last no isolated project  no person team work isolation successful outcome thread work lead inherently depend getting help helping member team role description in role help expand signaloid s application portfolio you explore wide rang

RuntimeError: The size of tensor a (549) must match the size of tensor b (512) at non-singleton dimension 1

In [180]:
cluster_skill = {}
for label in tqdm(cluster['cluster_graph'].unique()):
    skills = []
    for id in cluster.loc[cluster['cluster_graph'] == label]['id']:
        skills.extend(pipe(df.loc[df['id'] == id]['description'].values[0]))
        
    skill_extract = [{'entity': entry['entity'], 'word': entry['word']} for entry in skills]
    cluster_skill[label] = skill_extract


  0%|          | 0/15 [00:00<?, ?it/s]


RuntimeError: The size of tensor a (717) must match the size of tensor b (512) at non-singleton dimension 1

In [163]:
cluster_skill[9]

[{'entity': 'B-TECHNOLOGY', 'word': 'electronics'},
 {'entity': 'B-TECHNOLOGY', 'word': 'hardware'},
 {'entity': 'B-BUS', 'word': 'construction'},
 {'entity': 'B-TECHNOLOGY', 'word': 'electronics'},
 {'entity': 'B-BUS', 'word': 'project'},
 {'entity': 'B-BUS', 'word': 'financing'},
 {'entity': 'B-BUS', 'word': 'propulsion'},
 {'entity': 'B-BUS', 'word': 'av'},
 {'entity': 'I-BUS', 'word': '##ion'},
 {'entity': 'I-BUS', 'word': '##ics'},
 {'entity': 'B-TECHNICAL', 'word': 'software'},
 {'entity': 'I-TECHNICAL', 'word': 'testing'},
 {'entity': 'B-SOFT', 'word': 'commercial'},
 {'entity': 'B-TECHNICAL', 'word': 'data'},
 {'entity': 'I-TECHNICAL', 'word': 'acquisition'},
 {'entity': 'B-TECHNOLOGY', 'word': 'control'},
 {'entity': 'I-TECHNOLOGY', 'word': 'system'},
 {'entity': 'B-TECHNOLOGY', 'word': 'hardware'},
 {'entity': 'B-SOFT', 'word': 'instrumentation'},
 {'entity': 'B-BUS', 'word': 'construction'},
 {'entity': 'B-TECHNICAL', 'word': 'troubles'},
 {'entity': 'I-TECHNICAL', 'word': '

In [146]:

pd.DataFrame.from_dict(cluster_skill.items()).to_csv("./csv_files/skills.csv", index=False)

In [202]:
def words_to_sentence(word_list):
    return " ".join(word_list)
def remove_words_with_numbers(word_list_str):
    """
    Takes a string representation of a list of words as input,
    removes any special characters from the words, and then removes any words that contain numbers.

    Args:
      word_list_str: A string representation of a list of words.

    Returns:
      The function `remove_words_with_numbers` returns a list of words without any special characters or
    numbers.
    """
    word_list = ast.literal_eval(word_list_str)
    word_list_without_special = [
        re.sub(r"[^a-zA-Z0-9\s]", "", word) for word in word_list
    ]
    word_list_without_numbers = [
        word for word in word_list_without_special if not re.search(r"\d", word)
    ]
    return word_list_without_numbers
df["description"] = df["description"].apply(
            lambda x: remove_words_with_numbers(x)
        )
df["description"] = df["description"].apply(words_to_sentence)

In [ ]:
inghui - once you have pipeline ready for HF you can also try ExtractiveQA models, here si syntax more or less:

oracle = pipeline(model="deepset/roberta-base-squad2")
oracle(question="Where do I live?", context="My name is Wolfgang and I live in Berlin")
{'score': 0.9191, 'start': 34, 'end': 40, 'answer': 'Berlin'}

approach would be similar - question would be "what are needed skills?"
context would be job secription
description*


Let me know if you need hand with it

generative model
https://huggingface.co/docs/transformers/main_classes/pipelines#natural-language-processing
